# Lab-table maker

This notebook has been superseeded by the program Lab-table maker (lab_table_maker.py). It is left as an example for using iPyWidgets (Jupyter Widgets). The "Copy" button does not work.

In [4]:
from io import StringIO
import json
from collections import defaultdict

from IPython.display import display, Markdown, Javascript, HTML
from ipywidgets import interact, widgets
import pandas as pd

In [2]:
group_header = "Grupa"


def read_table(input: str):
    converters = {
        group_header: str,
        "ID": str,
        "JMBAG": str,
        "Prezime": str,
        "Ime": str,
    }
    df = pd.read_csv(
        StringIO(input),
        sep="\t",
        header=None,
        names=converters.keys(),
        converters=converters,
        index_col=False,
    )
    return df

def split_groups(df):
    room_to_groups = defaultdict(lambda: [])
    for group, d in df.groupby(group_header):
        room = group.split(" ")[-1]
        room_to_groups[room].append((group, d.drop(columns=[group_header])))
    return room_to_groups


def process_input(input: str):
    df = read_table(input)
    df["Bodovi"] = ""
    df["Komentar"] = " " * 64
    df.set_index('ID', inplace=True)
    df = df.rename_axis(None)
    room_to_groups = split_groups(df)
    output_parts = ["""
                    <html>
                    <style>
                    @media all { .page-break { display: none; } }
                    @media print { .page-break { display: block; page-break-before: always; } }
                    table, td, th {
                        border: 0.05rem solid #000;
                        border-collapse: collapse;
                        padding: 0.25rem;
                    }
                    </style>
                    <body>"""]
    for room, groups in room_to_groups.items():
        output_parts.append(
            '<div class="page-break"></div>'
            + f"\n\n<h2> {room} </h2>"
        )
        for group, df in groups:
            dfs = df.style.set_properties(**{'text-align': 'left'})
            dfs = dfs.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
            dfs.set_properties(subset=['Komentar'], **{'width': '24em'})
            output_parts.append(
                '<div class="page-break"></div>'
                + f"\n\n<h3> {group} </h3>"
                + dfs.to_html()
            )
    output_parts.append("\n</body>\n</html>")
    return "\n".join(output_parts)

In [3]:
input_textarea = widgets.Textarea(
    value="",
    description="Input:",
    layout=widgets.Layout(height="200px", width="auto"),
)
output_textarea = widgets.Textarea(
    value="",
    description="Output:",
    layout=widgets.Layout(height="200px", width="auto"),
)
submit_button = widgets.Button(description="Submit", icon="run", button_style="success")
copy_button = widgets.Button(description="Copy", icon="copy", button_style="success")


def on_submit_button_clicked(b):
    try:
        output_textarea.value = process_input(input_textarea.value)
        display(Markdown(output_textarea.value))
    except Exception as e:
        output_textarea.value = str(e)


def on_copy_button_clicked(b):
    try:
        output = widgets.Output(layout=widgets.Layout(display="none"))
        copy_js = Javascript(
            f"navigator.clipboard.writeText({json.dumps(output_textarea.value)})"
        )
        output.clear_output()
        output.append_display_data(copy_js)
    except Exception as e:
        output_textarea.value = str(e)

display(input_textarea, submit_button, output_textarea, copy_button)
submit_button.on_click(on_submit_button_clicked)
copy_button.on_click(on_copy_button_clicked)    

Textarea(value='', description='Input:', layout=Layout(height='200px', width='auto'))

Button(button_style='success', description='Submit', icon='run', style=ButtonStyle())

Textarea(value='', description='Output:', layout=Layout(height='200px', width='auto'))

Button(button_style='success', description='Copy', icon='copy', style=ButtonStyle())